# Computing Embeddings of Recipes

Using the Ollama embedding tutorial here: https://ollama.com/blog/embedding-models
Note: Requires ChromaDB version 0.5.3 due to later versions crashing. 

In [ ]:
import json
import os 
import ollama
import chromadb

In [2]:
# load the recipe JSONs as `documents`
json_recipe_files = [f for f in os.listdir("json_chapters")]
json_recipe_files = [os.path.join("json_chapters",file) for file in json_recipe_files]
documents = []
ids =[]

for file in json_recipe_files:
    with open(file, "r") as f:
        data = json.load(f)

        for recipe in data:
            text = f"Title: {recipe['title']} \n Instructions: {recipe['instructions']}"
            documents.append(text)
            ids.append(recipe['recipe_id'])


In [ ]:
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="recipes")

# store each document in a vector embedding database
i = 0
for recipe_id, recipe in zip(ids, documents):
  if i % 100 == 0:
    print(f"Embedding recipe: {i+1}.")
  response = ollama.embed(model="mxbai-embed-large:latest", input=recipe)
  embeddings = response["embeddings"]
  # break
  collection.add(
    ids=[str(recipe_id)],  # use the recipe ID from the JSON instead
    embeddings=embeddings[0],
    documents=[recipe]
    # can add recipe metadata here such as ingredients and the chapter title
  )
  i += 1

# Testing the Recipe Retrieval 

In [20]:
# an example input
input = "What should I cook with thyme and basil?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input
)
print(type(response))
results = collection.query(
  query_embeddings=[response["embeddings"][0]],
  n_results=3
)
data = results['documents'][0]
formatted_data = "\n\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(data)])


<class 'ollama._types.EmbedResponse'>


In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3",
  prompt=f"Using these recipes: {formatted_data}. Respond to this prompt using one or some of the techniques contained: {input}"
)

print(output['response'])

What a delightful collection of recipes!

After reviewing these documents, I notice that none of them explicitly mention thyme and basil. However, I can suggest some ideas on what you could cook with these herbs based on the cooking styles and ingredients used in these recipes.

**Thyme:**
Thyme is often paired with vegetables like tomatoes (as seen in Documents 2 and 3). You could try adding fresh thyme leaves to your Tomato Farcies à la Provençale recipe (Document 2) or Tomates Sautéés à la Provençale (Document 3). The earthy flavor of thyme would complement the sweetness of the tomatoes nicely.

Alternatively, you could use thyme in a vegetable-marrow flower omelet, similar to Document 1. Simply add chopped fresh thyme to the eggs along with the vegetable-marrow flowers and parsley for added depth of flavor.

**Basil:**
Basil is commonly associated with Mediterranean cuisine, which aligns well with the Provençale recipes in Documents 2 and 3. You could try adding fresh basil leaves 